# Proyecto Final
## Minería de Datos
### Nombres
* Kevin Arturo Bolivar Ramirez. 2131931.
* Yeisson Fabian Gualdron Vivas 2127790.

## Librerias 

In [1]:
from sys import stdin
import pandas as pd
import numpy as np
from random import randint
from sklearn.base import TransformerMixin
from sklearn.metrics import mean_squared_error
from sklearn.impute import KNNImputer

## 1. Datos Faltantes

Cargamos los dos archivos ".csv" y creamos dos dataframes respectivos


In [2]:
fileWineRed= pd.read_csv("winequality-red.csv",";")

fileWineWhite = pd.read_csv("winequality-white.csv",";")


In [3]:
fileWineRed.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [4]:
fileWineWhite.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6


Contamos los valores faltantes en cada columna por cada dataset

In [5]:
print("fileWineRed Missing values:\n",fileWineRed.isna().sum(),"\n")
print("fileWineWhite Missing values:\n",fileWineWhite.isna().sum(),"\n")

fileWineRed Missing values:
 fixed acidity           0
volatile acidity        0
citric acid             0
residual sugar          0
chlorides               0
free sulfur dioxide     0
total sulfur dioxide    0
density                 0
pH                      0
sulphates               0
alcohol                 0
quality                 0
dtype: int64 

fileWineWhite Missing values:
 fixed acidity           0
volatile acidity        0
citric acid             0
residual sugar          0
chlorides               0
free sulfur dioxide     0
total sulfur dioxide    0
density                 0
pH                      0
sulphates               0
alcohol                 0
quality                 0
dtype: int64 



Generar una nueva variable cualitativa que distinga entre el vino rojo y el vino blanco antes de unir los datos en un solo archivo.

In [6]:
fileWineWhite.columns

Index(['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar',
       'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density',
       'pH', 'sulphates', 'alcohol', 'quality'],
      dtype='object')

In [7]:
fileWineWhite = fileWineWhite.reindex(columns=['Variedad','fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar',
       'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density',
       'pH', 'sulphates', 'alcohol', 'quality'])
fileWineRed  = fileWineRed.reindex(columns=['Variedad','fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar',
       'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density',
       'pH', 'sulphates', 'alcohol', 'quality'])


In [8]:
FW = fileWineWhite.assign(Variedad = "W")

In [9]:
FR = fileWineRed.assign(Variedad = "R")

#### Crear un único conjunto de datos

In [10]:
VariablesBlanco = FW[['Variedad','fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar',
       'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density',
       'pH', 'sulphates', 'alcohol', 'quality']]
VariablesRojo = FR[['Variedad','fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar',
       'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density',
       'pH', 'sulphates', 'alcohol', 'quality']]
WineQuality = [VariablesBlanco,VariablesRojo]
Datos = pd.concat(WineQuality)
Datos

,Variedad,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,W,7.0,0.270,0.36,20.7,0.045,45.0,170.0,1.00100,3.00,0.45,8.8,6
1,W,6.3,0.300,0.34,1.6,0.049,14.0,132.0,0.99400,3.30,0.49,9.5,6
2,W,8.1,0.280,0.40,6.9,0.050,30.0,97.0,0.99510,3.26,0.44,10.1,6
3,W,7.2,0.230,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6
4,W,7.2,0.230,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1594,R,6.2,0.600,0.08,2.0,0.090,32.0,44.0,0.99490,3.45,0.58,10.5,5
1595,R,5.9,0.550,0.10,2.2,0.062,39.0,51.0,0.99512,3.52,0.76,11.2,6
1596,R,6.3,0.510,0.13,2.3,0.076,29.0,40.0,0.99574,3.42,0.75,11.0,6
1597,R,5.9,0.645,0.12,2.0,0.075,32.0,44.0,0.99547,3.57,0.71,10.2,5


In [11]:
#Paso adicional para unificar la enumeración de los registros.
Wine = Datos.to_numpy()
Data = pd.DataFrame(Wine)
Renombrar = {0 : 'Variedad',1 :'fixed acidity', 2 :'volatile acidity', 3 :'citric acid', 4 :'residual sugar',
       5 : 'chlorides',6 : 'free sulfur dioxide',7 :'total sulfur dioxide',8 :'density',
       9 :'pH',10 : 'sulphates',11 : 'alcohol',12 : 'quality' }
Data1 = Data.rename(columns = Renombrar)
Data1

,Variedad,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,W,7,0.27,0.36,20.7,0.045,45,170,1.001,3,0.45,8.8,6
1,W,6.3,0.3,0.34,1.6,0.049,14,132,0.994,3.3,0.49,9.5,6
2,W,8.1,0.28,0.4,6.9,0.05,30,97,0.9951,3.26,0.44,10.1,6
3,W,7.2,0.23,0.32,8.5,0.058,47,186,0.9956,3.19,0.4,9.9,6
4,W,7.2,0.23,0.32,8.5,0.058,47,186,0.9956,3.19,0.4,9.9,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6492,R,6.2,0.6,0.08,2,0.09,32,44,0.9949,3.45,0.58,10.5,5
6493,R,5.9,0.55,0.1,2.2,0.062,39,51,0.99512,3.52,0.76,11.2,6
6494,R,6.3,0.51,0.13,2.3,0.076,29,40,0.99574,3.42,0.75,11,6
6495,R,5.9,0.645,0.12,2,0.075,32,44,0.99547,3.57,0.71,10.2,5


Cambiar la variable categorica a numerica

In [12]:
Data2 = Data1.copy()
def cambiar(x):
    if x=='W':
        return 1
    if x=='R':
        return 2

In [13]:
Data2['Variedad']=Data2['Variedad'].apply(cambiar)
Data2

,Variedad,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,1,7,0.27,0.36,20.7,0.045,45,170,1.001,3,0.45,8.8,6
1,1,6.3,0.3,0.34,1.6,0.049,14,132,0.994,3.3,0.49,9.5,6
2,1,8.1,0.28,0.4,6.9,0.05,30,97,0.9951,3.26,0.44,10.1,6
3,1,7.2,0.23,0.32,8.5,0.058,47,186,0.9956,3.19,0.4,9.9,6
4,1,7.2,0.23,0.32,8.5,0.058,47,186,0.9956,3.19,0.4,9.9,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6492,2,6.2,0.6,0.08,2,0.09,32,44,0.9949,3.45,0.58,10.5,5
6493,2,5.9,0.55,0.1,2.2,0.062,39,51,0.99512,3.52,0.76,11.2,6
6494,2,6.3,0.51,0.13,2.3,0.076,29,40,0.99574,3.42,0.75,11,6
6495,2,5.9,0.645,0.12,2,0.075,32,44,0.99547,3.57,0.71,10.2,5


In [14]:
print("Valores Faltantes:\n",Data2.isna().sum(),"\n")

Valores Faltantes:
 Variedad                0
fixed acidity           0
volatile acidity        0
citric acid             0
residual sugar          0
chlorides               0
free sulfur dioxide     0
total sulfur dioxide    0
density                 0
pH                      0
sulphates               0
alcohol                 0
quality                 0
dtype: int64 



# Generar 10% de datos faltantes

In [15]:
def makeMissing(initialDF):
    newDF =  initialDF.copy()
    ten = int(newDF.shape[0]*newDF.shape[1]*0.1)
    print(ten)
    for i in range(ten):
        randomCol = newDF.columns[randint(0,12)]
        randomRow = randint(0,newDF.shape[0]-1)
        newDF[randomCol][randomRow] = np.nan
    return newDF

Nuevo Data Set

In [16]:
dfWQ = makeMissing(Data2) 

8446


<ipython-input-15-880575c09b7d>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  newDF[randomCol][randomRow] = np.nan
C:\Users\yeiso\anaconda3\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


Verificar el porcentaje de faltantes

In [17]:
A = dfWQ.size
round(8446/A*100,0)

10.0

Transformar la variable categorica a numérica

In [18]:
print("Valores Faltantes:\n",dfWQ.isna().sum(),"\n")

Valores Faltantes:
 Variedad                589
fixed acidity           597
volatile acidity        603
citric acid             592
residual sugar          633
chlorides               648
free sulfur dioxide     630
total sulfur dioxide    611
density                 619
pH                      574
sulphates               641
alcohol                 645
quality                 646
dtype: int64 



Guardamos el dataset con los valores faltantes

In [36]:
dfWQ.to_csv('persistentDataset.csv', index = False)

# Imputación de Datos


### Por la Moda

* https://stackoverflow.com/questions/25239958/impute-categorical-missing-values-in-scikit-learn

In [19]:
class DataFrameImputer(TransformerMixin):

       def __init__(self):
           """Impute missing values.

           Columns of dtype object are imputed with the most frequent value 
           in column.

           Columns of other types are imputed with mean of column.

           """
       def fit(self, X, y=None):

           self.fill = pd.Series([X[c].value_counts().index[0]
               if X[c].dtype == np.dtype('O') else X[c].mean() for c in X],
               index=X.columns)

           return self

       def transform(self, X, y=None):
           return X.fillna(self.fill)

WineIMP1= DataFrameImputer().fit_transform(dfWQ)
WineIMP1.isna().sum()

Variedad                0
fixed acidity           0
volatile acidity        0
citric acid             0
residual sugar          0
chlorides               0
free sulfur dioxide     0
total sulfur dioxide    0
density                 0
pH                      0
sulphates               0
alcohol                 0
quality                 0
dtype: int64

Evaluación del método

In [20]:
mean_squared_error(Data2,WineIMP1)

28.5146980406296

### Imputación por vecino más cercano KNN = 3.

In [21]:
imputador = KNNImputer (n_neighbors = 3)
WineImpKNN2 = imputador.fit_transform (dfWQ)
WineImpKNN2

array([[ 1.        ,  7.        ,  0.25      , ...,  0.45      ,
         8.8       ,  6.        ],
       [ 1.        ,  7.26666667,  0.3       , ...,  0.49      ,
         9.5       ,  6.        ],
       [ 1.        ,  8.1       ,  0.28      , ...,  0.46666667,
        10.1       ,  6.        ],
       ...,
       [ 2.        ,  6.3       ,  0.51      , ...,  0.67666667,
        11.        ,  6.        ],
       [ 2.        ,  5.9       ,  0.46      , ...,  0.71      ,
        10.2       ,  6.        ],
       [ 2.        ,  6.        ,  0.31      , ...,  0.66      ,
        11.        ,  6.        ]])

In [22]:
WineImp2 =pd.DataFrame(WineImpKNN2)
WineImp2

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,1.0,7.000000,0.250000,0.360000,20.7,0.045,45.0,114.333333,1.00100,3.00,0.450000,8.8,6.0
1,1.0,7.266667,0.300000,0.340000,1.6,0.049,14.0,132.000000,0.99400,3.30,0.490000,9.5,6.0
2,1.0,8.100000,0.280000,0.400000,6.9,0.050,30.0,125.333333,0.99510,3.26,0.466667,10.1,6.0
3,1.0,7.200000,0.213333,0.383333,8.5,0.058,47.0,129.333333,0.99560,3.19,0.400000,9.9,6.0
4,1.0,7.200000,0.230000,0.320000,8.5,0.058,47.0,186.000000,0.99560,3.19,0.400000,9.9,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6492,2.0,6.200000,0.600000,0.080000,5.1,0.090,32.0,44.000000,0.99490,3.45,0.613333,10.5,5.0
6493,2.0,5.900000,0.550000,0.100000,2.2,0.062,39.0,51.000000,0.99512,3.52,0.760000,11.2,6.0
6494,2.0,6.300000,0.510000,0.130000,2.3,0.076,29.0,40.000000,0.99574,3.42,0.676667,11.0,6.0
6495,2.0,5.900000,0.460000,0.120000,2.0,0.075,32.0,44.000000,0.99547,3.57,0.710000,10.2,6.0


In [23]:
WineIMP2 = WineImp2.rename(columns = Renombrar)
WineIMP2

,Variedad,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,1.0,7.000000,0.250000,0.360000,20.7,0.045,45.0,114.333333,1.00100,3.00,0.450000,8.8,6.0
1,1.0,7.266667,0.300000,0.340000,1.6,0.049,14.0,132.000000,0.99400,3.30,0.490000,9.5,6.0
2,1.0,8.100000,0.280000,0.400000,6.9,0.050,30.0,125.333333,0.99510,3.26,0.466667,10.1,6.0
3,1.0,7.200000,0.213333,0.383333,8.5,0.058,47.0,129.333333,0.99560,3.19,0.400000,9.9,6.0
4,1.0,7.200000,0.230000,0.320000,8.5,0.058,47.0,186.000000,0.99560,3.19,0.400000,9.9,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6492,2.0,6.200000,0.600000,0.080000,5.1,0.090,32.0,44.000000,0.99490,3.45,0.613333,10.5,5.0
6493,2.0,5.900000,0.550000,0.100000,2.2,0.062,39.0,51.000000,0.99512,3.52,0.760000,11.2,6.0
6494,2.0,6.300000,0.510000,0.130000,2.3,0.076,29.0,40.000000,0.99574,3.42,0.676667,11.0,6.0
6495,2.0,5.900000,0.460000,0.120000,2.0,0.075,32.0,44.000000,0.99547,3.57,0.710000,10.2,6.0


In [24]:
print("Valores Faltantes:\n",WineIMP2.isna().sum(),"\n")

Valores Faltantes:
 Variedad                0
fixed acidity           0
volatile acidity        0
citric acid             0
residual sugar          0
chlorides               0
free sulfur dioxide     0
total sulfur dioxide    0
density                 0
pH                      0
sulphates               0
alcohol                 0
quality                 0
dtype: int64 



Evaluación del método KNN = 3

In [25]:
mean_squared_error(Data2,WineIMP2)

15.384852096874456

### Imputación por vecino más cercano KNN = 10.

In [26]:
imputador = KNNImputer (n_neighbors = 10)
WineImpKNN3 = imputador.fit_transform (dfWQ)
WineImpKNN3

array([[ 1.   ,  7.   ,  0.218, ...,  0.45 ,  8.8  ,  6.   ],
       [ 1.   ,  7.11 ,  0.3  , ...,  0.49 ,  9.5  ,  6.   ],
       [ 1.   ,  8.1  ,  0.28 , ...,  0.504, 10.1  ,  6.   ],
       ...,
       [ 2.   ,  6.3  ,  0.51 , ...,  0.567, 11.   ,  6.   ],
       [ 2.   ,  5.9  ,  0.338, ...,  0.71 , 10.2  ,  5.9  ],
       [ 2.   ,  6.   ,  0.31 , ...,  0.66 , 11.   ,  6.   ]])

In [27]:
WineImp3 =pd.DataFrame(WineImpKNN3)
WineImp3

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,1.0,7.00,0.218,0.360,20.70,0.045,45.0,140.6,1.00100,3.00,0.450,8.8,6.0
1,1.0,7.11,0.300,0.340,1.60,0.049,14.0,132.0,0.99400,3.30,0.490,9.5,6.0
2,1.0,8.10,0.280,0.400,6.90,0.050,30.0,158.9,0.99510,3.26,0.504,10.1,6.0
3,1.0,7.20,0.225,0.367,8.50,0.058,47.0,136.4,0.99560,3.19,0.400,9.9,6.0
4,1.0,7.20,0.230,0.320,8.50,0.058,47.0,186.0,0.99560,3.19,0.400,9.9,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6492,2.0,6.20,0.600,0.080,6.41,0.090,32.0,44.0,0.99490,3.45,0.529,10.5,5.0
6493,2.0,5.90,0.550,0.100,2.20,0.062,39.0,51.0,0.99512,3.52,0.760,11.2,6.0
6494,2.0,6.30,0.510,0.130,2.30,0.076,29.0,40.0,0.99574,3.42,0.567,11.0,6.0
6495,2.0,5.90,0.338,0.120,2.00,0.075,32.0,44.0,0.99547,3.57,0.710,10.2,5.9


In [28]:
WineIMP3 = WineImp3.rename(columns = Renombrar)
WineIMP3

,Variedad,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,1.0,7.00,0.218,0.360,20.70,0.045,45.0,140.6,1.00100,3.00,0.450,8.8,6.0
1,1.0,7.11,0.300,0.340,1.60,0.049,14.0,132.0,0.99400,3.30,0.490,9.5,6.0
2,1.0,8.10,0.280,0.400,6.90,0.050,30.0,158.9,0.99510,3.26,0.504,10.1,6.0
3,1.0,7.20,0.225,0.367,8.50,0.058,47.0,136.4,0.99560,3.19,0.400,9.9,6.0
4,1.0,7.20,0.230,0.320,8.50,0.058,47.0,186.0,0.99560,3.19,0.400,9.9,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6492,2.0,6.20,0.600,0.080,6.41,0.090,32.0,44.0,0.99490,3.45,0.529,10.5,5.0
6493,2.0,5.90,0.550,0.100,2.20,0.062,39.0,51.0,0.99512,3.52,0.760,11.2,6.0
6494,2.0,6.30,0.510,0.130,2.30,0.076,29.0,40.0,0.99574,3.42,0.567,11.0,6.0
6495,2.0,5.90,0.338,0.120,2.00,0.075,32.0,44.0,0.99547,3.57,0.710,10.2,5.9


In [29]:
print("Valores Faltantes:\n",WineIMP3.isna().sum(),"\n")

Valores Faltantes:
 Variedad                0
fixed acidity           0
volatile acidity        0
citric acid             0
residual sugar          0
chlorides               0
free sulfur dioxide     0
total sulfur dioxide    0
density                 0
pH                      0
sulphates               0
alcohol                 0
quality                 0
dtype: int64 



Evaluación del KNN = 10

In [33]:
mean_squared_error(Data2,WineIMP3)

13.212375213771043